## Análise Exploratória de uma base de Crédito

### A base de dados que vamos explorar tem as seguintes características:

 - 3 variáveis independentes:
            - income (valor do salário)
            - age (idade)
            - loan (valor do empréstimo)
            
 - 1 variável dependente:
            - default (representa se o cliente conseguiu ou não a liberação do cérdito)

### Carregando a Base de Dados
#### Primeiro precisamos importar a biblioteca pandas e em seguida ler a base de dados:

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('credit_data.csv')

In [3]:
# Espiando as primeiras 5 linhas da base de dados, podemos ver as variáveis e também algumas informações de tipos dos dados
df.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [4]:
# Podemos observar que os nomes de algumas variáveis estão com caracteres especiais como #, assim, com intuito de evitar problemas 
# em nosso modelo vamos renomear essas colunas
df.columns

Index(['i#clientid', 'income', 'age', 'loan', 'c#default'], dtype='object')

In [5]:
# renomeando as colunas:
df.rename(columns={'i#clientid': 'clientid', 'c#default': 'default'}, inplace = True)


In [6]:
# Agora sim, bem melhor não acha?
df.columns

Index(['clientid', 'income', 'age', 'loan', 'default'], dtype='object')

#### Que tal chegar os tipos de dados em nosso conjunto? Também podemos aproveitar e verificar se existem valores missing.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   clientid  2000 non-null   int64  
 1   income    2000 non-null   float64
 2   age       1997 non-null   float64
 3   loan      2000 non-null   float64
 4   default   2000 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 78.2 KB


#### Podemos observar que temos dois tipos de dados em nossa base, inteiros e float e também podemos ver que na coluna "*age*" faltam alguns dados, interessate não?

In [9]:
# Podemos aproveitar e averiguar alguns dados estatísticos do nosso dataset
df.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


## Tratamento de valores inconsistentes

Ao aplicar o método describe() do pandas, podemos observar que existem idades com valores negativos. Mas como assim? Precisamos corrigir isso para não termos poblemas com nosso modelo mais pra frente

In [15]:
# Vamos identficar os clientes com idades negativas:
df.query('age < 0')
#df.loc[df['age'] < 0]

,clientid,income,age,loan,default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


In [16]:
# O primeiro tratamento que poderíamos aplicar é deletar os dados de valores negativos, para isso criaremos um novo objeto paea
# receber uma cópia do nosso dataset para não danificarmos os dados originais, afinal, é somente para tírulo de conhecimento.
df2 = df

In [17]:
df2.head()

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [19]:
# identificamos os indices das linhas que contém os valores negativos e passamos o método "drop" para excluir os dados.
df2 = df2.drop([15, 21, 26])

In [20]:
# Podemos ver que não existem mais idades negativas, visto que a query não retornou nada
df2.query('age < 0')

,clientid,income,age,loan,default


In [24]:
# Outra forma de tartar esse dados negativos é substituindo-os pela média das idades
df.age.mean()

40.80755937840458

In [28]:
# Vamos tirar a média das idades excluindo aqueles 3 valores negativos
df['age'][df.age > 0].mean()

40.92770044906149

In [30]:
# Criamos um objeto para receber a média dos valores de idade exlcuindo os valores negativos
media = df['age'][df.age > 0].mean()

In [31]:
media

40.92770044906149

In [34]:
# Atribuindo o valor de media aos valores negativos:
df.iloc[[15, 21, 26]] = media

In [35]:
df.iloc[[15, 21, 26]]

,clientid,income,age,loan,default
15,40.9277,40.9277,40.9277,40.9277,40.9277
21,40.9277,40.9277,40.9277,40.9277,40.9277
26,40.9277,40.9277,40.9277,40.9277,40.9277


### Tratando Valores faltantes

In [36]:
df.columns

Index(['clientid', 'income', 'age', 'loan', 'default'], dtype='object')

In [39]:
df['income'][pd.isnull(df.income)]

Series([], Name: income, dtype: float64)

In [41]:
df.loc[pd.isnull(df.age)]

,clientid,income,age,loan,default
28,29.0,59417.805406,NaN,2082.625938,0.0
30,31.0,48528.852796,NaN,6155.784670,0.0
31,32.0,23526.302555,NaN,2862.010139,0.0


### Podemos tratar os dados faltantes diretamente com a biblioteca sckit Learn. Para isso iremos dividir nosso dataset em dois, um eceberá somente as variáveis previsoras e o outro receberá nossa classe

In [62]:
previsores = df.iloc[:, 1:4].values

In [64]:
previsores

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [65]:
classe = df.iloc[:, 4].values

In [66]:
classe

array([0., 0., 0., ..., 1., 0., 0.])

#### Vejamos como tratar os dados faltantes direto com o Scikit Learn

In [50]:
from sklearn.impute import SimpleImputer

In [58]:
import numpy as np
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [68]:
imputer = imputer.fit(previsores[:, 0:3])
previsores[:, 0:3] = imputer.transform(previsores[:,0:3])

### Escalonamento de variáveis com SkLearn

In [52]:
# iremos utilizar a técnica da Padronização, que é maais robusta e portanto menos sensíveis aos outliers
from sklearn.preprocessing import StandardScaler

In [69]:
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [70]:
previsores

array([[ 1.44913211,  1.36538005,  1.20471111],
       [-0.75194964,  0.54265932,  0.69874452],
       [ 0.83620364,  1.67417101,  1.17662679],
       ...,
       [-0.06568543, -0.97448606,  0.35680621],
       [-0.10419945,  1.73936652, -0.92307354],
       [ 1.67663095,  1.14917551,  0.96590308]])